# 라이브러리 import

In [17]:
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import re
from tqdm import tqdm
from selenium.common.exceptions import NoSuchElementException


# 함수 정의

In [32]:
def get_url_list():
    with open("../data/job_fix_list.csv", encoding='cp949') as f:
        urls = f.readlines()

#     url_list = []

#     for url in urls:
#         url_spl = url.rstrip("\n").split(",")

#         job = ",".join(url_spl[:-1]).strip('""')
#         url = url_spl[-1]

#         url_list.append([job, url])

    url_list = [url.rstrip("\n") for url in urls]
    
    return url_list

def get_job_description(driver, url):
    
    # Job Description 을 담을 딕셔너리 생성
    res = {}
    
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            
            # url 접속하기
            driver.get(url)

            # 화면 비율을 축소해서 모든 데이터 보이게 만들기
            driver.execute_script("document.body.style.zoom='10%'")

            time.sleep(5)

            # url 에서 index 값 추출
            res['id'] = url.split("/")[-1]

            # JobHeader 클래스 가져와서 title, company, tags 추출
            job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
            title = job_header.find_element(By.TAG_NAME, 'h2')
            company = job_header.find_element(By.TAG_NAME, 'a')
            location = job_header.find_element(By.CLASS_NAME, 'JobHeader_mobileLocationContainer__DyxUQ')
            tags = job_header.find_element(By.CLASS_NAME, 'Tags_tagsClass__mvehZ')

            # res 딕셔너리에 저장
            res['title'] = title.text
            res['company'] = company.text
            res['location'] = location.text.split(".")

            # Tag 리스트 가져와서 저장
            tag_list = list(map(str, tags.text.split('#')))
            tag_list = tag_list[1:]
            res['tags'] = tag_list

            # JobDescription 클래스 가져와서 header, contents 추출
            jd = driver.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__VWfcb')
            jd_header = jd.find_elements(By.TAG_NAME, 'h6')
            jd_contents = jd.find_elements(By.TAG_NAME, 'p')

            for i in range(1, len(jd_header)+1):
                title = jd_header[i-1].text
                content = jd_contents[i].text.split("\n")

                if title == '자격요건':
                    res['qualifications'] = content
                elif title == '우대사항':
                    res['preferred'] = content
                elif title == '혜택 및 복지':
                    res["benefits"] = content
                elif title == '기술스택 ・ 툴':
                    res["skills"] = content


            jwp = driver.find_element(By.CLASS_NAME, "JobWorkPlace_className__ra6rp")
            jwp_bodies = jwp.find_elements(By.CLASS_NAME, 'body')

            if len(jwp_bodies) < 2:
                workplace = ""
            else:
                workplace = jwp_bodies[1].text

            res['workplace'] = workplace
        except NoSuchElementException:
            print("존재하지 않는 페이지 입니다.")
    else:
        print("존재하지 않는 페이지 입니다.")
        print("res = ", res)
    
    return res

# 크롤링 실행

In [34]:
driver = webdriver.Chrome()

# url 가져오기
url_list = get_url_list()

total = []

for i in tqdm(range(2000, 4000)):
    res = get_job_description(driver, url_list[i])
    
    if len(res) == 0:
        print("존재하지 않는 페이지 입니다.")
        continue
    else:
        total.append(res)

driver.quit()

  0%|          | 2/2000 [00:22<6:16:27, 11.31s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0074A813+48355]
	(No symbol) [0x006DC4B1]
	(No symbol) [0x005E5358]
	(No symbol) [0x005CD293]
	(No symbol) [0x0062E37B]
	(No symbol) [0x0063C473]
	(No symbol) [0x0062A536]
	(No symbol) [0x006082DC]
	(No symbol) [0x006093DD]
	GetHandleVerifier [0x009AAABD+2539405]
	GetHandleVerifier [0x009EA78F+2800735]
	GetHandleVerifier [0x009E456C+2775612]
	GetHandleVerifier [0x007D51E0+616112]
	(No symbol) [0x006E5F8C]
	(No symbol) [0x006E2328]
	(No symbol) [0x006E240B]
	(No symbol) [0x006D4FF7]
	BaseThreadInitThunk [0x76D87D59+25]
	RtlInitializeExceptionChain [0x77E4B74B+107]
	RtlClearBits [0x77E4B6CF+191]


In [ ]:
url_list = get_url_list()

print(len(url_list))

5462


# 크롤링 결과 및 확인

In [35]:
print(total[0])
for t in total:
    print(t)


{'id': '88428', 'title': 'RPA SM/SI', 'company': '아임잇', 'location': [''], 'tags': ['퇴사율5%이하', '50명이하', '설립4~9년', '워크샵', '식비', '음료', '생일선물', '건강검진', 'IT, 컨텐츠'], 'qualifications': ['• 지원자격', '  경력 : 무관 (신입도 지원 가능)'], 'preferred': ['• 컴퓨터/시스템공학', '• 정보처리기사', '• RPA 솔루션 유경험자', '• MS Office(엑셀 함수, VBA) 경험자'], 'benefits': ['• 퇴직연금, 장기근속자 포상, 퇴직금, 직책수당, 4대 보험', '• 건강검진, 각종 경조사 지원', '• 명절선물/귀향비, 생일선물/파티', '• 워크샵, 신입사원교육(OJT), 직무능력향상교육, 도서구입비지원, 식비 지원, 음료제공', '• 연차, 여름휴가, 경조휴가제, 반차, Refresh휴가, 근로자의 날 휴무, 포상휴가'], 'skills': ['Java', 'WebSphere MQ'], 'workplace': '서울특별시 금천구 가산디지털1로'}
{'id': '88428', 'title': 'RPA SM/SI', 'company': '아임잇', 'location': [''], 'tags': ['퇴사율5%이하', '50명이하', '설립4~9년', '워크샵', '식비', '음료', '생일선물', '건강검진', 'IT, 컨텐츠'], 'qualifications': ['• 지원자격', '  경력 : 무관 (신입도 지원 가능)'], 'preferred': ['• 컴퓨터/시스템공학', '• 정보처리기사', '• RPA 솔루션 유경험자', '• MS Office(엑셀 함수, VBA) 경험자'], 'benefits': ['• 퇴직연금, 장기근속자 포상, 퇴직금, 직책수당, 4대 보험', '• 건강검진, 각종 경조사 지원', '• 명절선물/귀향비, 생일선물/파티', '• 워크샵, 신입사원교육(OJT),

# 결과 csv 파일로 저장하기

In [22]:
import csv

with open('C:\codes\playground-data\crawler\output/wanted_crawling_output_8_3001_4000sam.csv', 'w', encoding='UTF-8') as f:
# with open('C:\codes\playground-data\crawler\output/EXAMPLE.csv', 'w', encoding='UTF-8') as f:
    # csv 파일의 헤더 설정
    header = ['id', 'title', 'company', 'location', 'tags', 'qualifications', 'preferred', 'benefits', 'skills', 'workplace']

    # 헤더 적용
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    
    # 헤더에 맞게 딕셔너리 값 입력
    for jd in total:
        writer.writerow(jd)

# 전처리 (진행중)

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


exception_list = ['를', '를', '를', '을', '을', '있거나', '있으신', '갖추신', '경험이', '이상의', '에', '에', '에', '에', '에', '이용한', '에', '에', '에', '준하는', '에', '있으신', '에', '에', '에', '중심적인', '에', '에', '이해를', '에', '대한', '및', '가지고', '전반적인', '에']

# 텍스트 정의
contents = "• 6년 이상의 백엔드 개발 경력이 있거나 그에 준하는 실력을 갖추신 분\n• JavaScript / TypeScript 를 이용한 개발에 능숙하신 분\n• 웹 서비스에 대한 전반적인 이해를 갖추신 분\n• RDB(Postgres) 및 ORM에 대한 전반적인 이해를 갖추신 분\n• 도커, 쿠버네티스 등 컨테이너 기반의 서비스에 대한 경험이 있으신 분\n• 1년 이상의 NestJS 실무 프로젝트 개발 경험이 있으신 분\n• 모듈화, 테스트, 유지보수가 용이한 코드에 대한 이해를 갖추고 계신 분\n• 기술 중심이 아닌 유저 중심적인 태도를 가지고 계신 분\n• 배움에 대한 열정을 갖고, 지속적이고 효과적으로 피드백을 주고 받는 분"

# 특수문자 제거
contents = re.sub(r'[^\w\s]', '', contents)

# 단어 토큰화
tokens = word_tokenize(contents)

# 품사 태깅
tagged_words = pos_tag(tokens)

# 명사 추출 (동사 및 불용어 제외)
nouns = [word for word, pos in tagged_words if pos.startswith('N') and not pos.startswith('V') and word.lower() not in exception_list]

# 중복 제거
nouns = list(set(nouns))

# 결과 출력
print(nouns)